In [1]:
import pandas as pd

data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
#data = data.sample(frac=0.1)
data = data.fillna(True)
data.HIREDT = data.HIREDT.apply(pd.to_datetime, format = "%m/%d/%y")
data['daysexperience'] = (pd.to_datetime('10/7/23', format = "%m/%d/%y") - data.HIREDT ).dt.days
data_num = ['RATE', 'HRSWKD', 'daysexperience']
data_drop = ['AGY', 'HIREDT']#, 'multiple_full_time_jobs', 'combined_multiple_jobs']
data.drop(data_drop, axis=1, inplace=True)

important_features = ['JC.TITLE', 'JOBCLASS', 'EMPTYPE', 'NAME', 'HRSWKD', 'daysexperience']


In [2]:
#data = data[important_features + ['ANNUAL']]
data_to_onehot = ['NAME', 'MI', 'JOBCLASS', 'JC.TITLE', 'RACE', 'SEX', 'EMPTYPE', 'STATENUM', 'multiple_full_time_jobs', 'combined_multiple_jobs']
#data_to_onehot = list(set(important_features) & set(data_to_onehot))

In [3]:
data.isna().sum()

NAME                       0
MI                         0
JOBCLASS                   0
JC.TITLE                   0
RACE                       0
SEX                        0
EMPTYPE                    0
RATE                       0
HRSWKD                     0
ANNUAL                     0
STATENUM                   0
multiple_full_time_jobs    0
combined_multiple_jobs     0
daysexperience             0
dtype: int64

In [4]:
data.nunique()

NAME                          113
MI                             27
JOBCLASS                     1666
JC.TITLE                     1608
RACE                            6
SEX                             2
EMPTYPE                         9
RATE                          169
HRSWKD                         47
ANNUAL                      28485
STATENUM                   112582
multiple_full_time_jobs         1
combined_multiple_jobs          1
daysexperience               5684
dtype: int64

In [5]:
data

,NAME,MI,JOBCLASS,JC.TITLE,RACE,SEX,EMPTYPE,RATE,HRSWKD,ANNUAL,STATENUM,multiple_full_time_jobs,combined_multiple_jobs,daysexperience
0,TEXAS HIGHER EDUCATION COORDINATING BOARD ...,C,7040,PROJECT MANAGER ...,BLACK,FEMALE,URF - UNCLASSIFIED REGULAR FULL-TIME,0.0000,40.0,75000.00,32448,True,True,310
1,SENATE ...,,7104,LEGISLATIVE PROFESSIONAL ...,WHITE,FEMALE,URF - UNCLASSIFIED REGULAR FULL-TIME,0.0000,41.0,65199.96,37375,True,True,6820
2,"COMPTROLLER OF PUBLIC ACCOUNTS, JUDICIARY SECT...",J,JD25,"JUDGE, RETIRED ...",WHITE,MALE,URP - UNCLASSIFIED REGULAR PART-TIME,75.9615,29.0,114549.84,59115,True,True,1344
3,SCHOOL FOR THE BLIND AND VISUALLY IMPAIRED ...,T,7354,LIBRARY ASSISTANT II ...,WHITE,MALE,CRP - CLASSIFIED REGULAR PART-TIME,0.0000,20.0,21865.68,79455,True,True,8766
4,SCHOOL FOR THE BLIND AND VISUALLY IMPAIRED ...,T,154,ADMINISTRATIVE ASSISTANT III ...,WHITE,MALE,CRP - CLASSIFIED REGULAR PART-TIME,0.0000,20.0,20294.76,79455,True,True,8766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112588,STATE AUDITOR'S OFFICE ...,H,AUD2,AUDITOR II ...,WHITE,MALE,URF - UNCLASSIFIED REGULAR FULL-TIME,0.0000,40.0,50000.04,1576395,True,True,279
112589,STATE AUDITOR'S OFFICE ...,T,AUD2,AUDITOR II ...,WHITE,MALE,URF - UNCLASSIFIED REGULAR FULL-TIME,0.0000,40.0,54000.00,1576407,True,True,279
112590,STATE AUDITOR'S OFFICE ...,H,AUD2,AUDITOR II ...,WHITE,MALE,URF - UNCLASSIFIED REGULAR FULL-TIME,0.0000,40.0,50000.04,1576554,True,True,279
112591,STATE AUDITOR'S OFFICE ...,N,AUD4,AUDITOR IV ...,HISPANIC,FEMALE,URF - UNCLASSIFIED REGULAR FULL-TIME,0.0000,40.0,68000.04,1577374,True,True,279


In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
regressor = MLPRegressor(random_state=1, max_iter=500)

transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'infrequent_if_exist'), data_to_onehot),
    remainder = 'passthrough',
)
pipe = Pipeline([('transformer', transformer), ('regressor', regressor)])

transformed = transformer.fit_transform(data)
transformed_df = pd.DataFrame(
    transformed, 
    columns=transformer.get_feature_names()
)
transformed_df

In [7]:
#data_onehot = pd.get_dummies(data, columns=data_to_onehot)

In [8]:
from sklearn.model_selection import train_test_split
X = data.drop('ANNUAL', axis=1)
y = data.ANNUAL

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

outliers_level = 0.05/100
d01 = y_train.quantile(q=outliers_level)
d99 = y_train.quantile(q=1-outliers_level)

outliers = y_train.index[(y_train < d01) | (y_train > d99)].tolist()

X_train.drop(outliers, inplace=True)
y_train.drop(outliers, inplace=True)

In [9]:
pipe.fit(X_train, y_train)

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['NAME', 'MI', 'JOBCLASS',
                                                   'JC.TITLE', 'RACE', 'SEX',
                                                   'EMPTYPE', 'STATENUM',
                                                   'multiple_full_time_jobs',
                                                   'combined_multiple_jobs'])])),
                ('regressor', MLPRegressor(max_iter=500, random_state=1))])

In [10]:
y_pred = pipe.predict(X_test)

In [11]:
pipe.score(X_test, y_test)

0.7461467914865298

In [12]:
res = pd.DataFrame(y_test)
res['pred'] = y_pred
res['diff'] = res.pred - res.ANNUAL
res['diff'] = res['diff'].apply(abs)

In [13]:
res['diff'].mean()

6290.188792893038

In [14]:
from sklearn.metrics import r2_score
r2_score(y_pred, y_test)

0.6297293979571377